# imports

In [20]:
# !pip install z3-solver

In [61]:
import xgboost as xgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from z3 import *
from xgboost import XGBClassifier
from pmlb import fetch_data

set_option(rational_to_decimal=True)

# model

In [2]:
from z3 import *
import numpy as np


class XGBoostExplainer:
    def __init__(self, model, data):
        self.model = model
        self.data = data.values
        self.columns = data.columns
        self.max_categories = 2

        set_option(rational_to_decimal=True)
        self.categoric_features = self.get_categoric_features(self.data)
        self.T_model = self.model_trees_expression(self.model)
        self.T = self.T_model

    def explain(self, instance, reorder="asc"):
        self.I = self.instance_expression(instance)
        self.D, self.D_add = self.decision_function_expression(self.model, [instance])
        return self.explain_expression(self.I, And(self.T, self.D_add), self.D, self.model, reorder)

    def explain_prob(self, instance, reorder="asc", threshold_margin=0):
        self.I = self.instance_expression(instance)
        self.D, self.D_add = self.decision_function_expression(self.model, [instance])
        return self.explain_expression_prob(self.I, And(self.T, self.D_add), self.D, self.model, reorder, threshold_margin)

    def get_categoric_features(self, data: np.ndarray):
        categoric_features = []
        for i in range(data.shape[1]):
            feature_values = data[:, i]
            unique_values = np.unique(feature_values)
            if len(unique_values) <= self.max_categories:
                categoric_features.append(self.columns[i])

        return categoric_features

    def feature_constraints(self, constraints=[]):
        """TODO
        esperado receber limites das features pelo usuário
        formato previso: matriz/dataframe [feaature, min/max, valor]
        constraaint_expression = "constraaint_df_to_feature()"
        """
        return

    def model_trees_expression(self, model):
        df = model.get_booster().trees_to_dataframe()
        if model.get_booster().feature_names == None:
            feature_map = {f"f{i}": col for i, col in enumerate(self.columns)}
            df["Feature"] = df["Feature"].replace(feature_map)

        df["Split"] = df["Split"].round(4)
        self.booster_df = df
        class_index = 0  # if model.n_classes_ == 2:
        all_tree_formulas = []

        for tree_index in df["Tree"].unique():
            tree_df = df[df["Tree"] == tree_index]
            o = Real(f"o_{tree_index}_{class_index}")

            if len(tree_df) == 1 and tree_df.iloc[0]["Feature"] == "Leaf":
                leaf_value = tree_df.iloc[0]["Gain"]
                all_tree_formulas.append(And(o == leaf_value))
                continue
            path_formulas = []

            def get_conditions(node_id):
                conditions = []
                current_node = tree_df[tree_df["ID"] == node_id]
                if current_node.empty:
                    return conditions

                parent_node = tree_df[
                    (tree_df["Yes"] == node_id) | (tree_df["No"] == node_id)
                ]
                if not parent_node.empty:
                    parent_data = parent_node.iloc[0]
                    feature = parent_data["Feature"]
                    split_value = parent_data["Split"]
                    x = Real(feature)
                    if parent_data["Yes"] == node_id:
                        conditions.append(x < split_value)
                    else:
                        conditions.append(x >= split_value)
                    conditions = get_conditions(parent_data["ID"]) + conditions

                return conditions

            for _, node in tree_df[tree_df["Feature"] == "Leaf"].iterrows():
                leaf_value = node["Gain"]
                leaf_id = node["ID"]
                conditions = get_conditions(leaf_id)
                path_formula = And(*conditions)
                implication = Implies(path_formula, o == leaf_value)
                path_formulas.append(implication)

            all_tree_formulas.append(And(*path_formulas))
        return And(*all_tree_formulas)

    def get_init_value(self, model, x, estimator_variables):
        estimator_pred = Solver()
        estimator_pred.add(self.I)
        estimator_pred.add(self.T)
        if estimator_pred.check() == sat:
            solvermodel = estimator_pred.model()
            total_sum = sum(
                float(solvermodel.eval(var).as_fraction()) for var in estimator_variables
            )
        else:
            total_sum = 0
            print("estimator error")
        self.predicted_margin = model.predict(x, output_margin=True)[0]
        init_value = self.predicted_margin - total_sum
        self.init_value = init_value
        return init_value

    def decision_function_expression(self, model, x):
        n_classes = 1 if model.n_classes_ <= 2 else model.n_classes_
        predicted_class = model.predict(x)[0]
        self.predicted_class = predicted_class
        n_estimators = int(len(model.get_booster().get_dump()) / n_classes)
        estimator_variables = [Real(f"o_{i}_0") for i in range(n_estimators)] # _0 only for binary classification
        self.estimator_variables = estimator_variables
        init_value = self.get_init_value(model, x, estimator_variables)
        # print("init:", round(init_value, 2))

        equation_list = []

        estimator_sum = Real("estimator_sum")
        equation_o = estimator_sum == Sum(estimator_variables)
        equation_list.append(equation_o)

        decision = Real("decision")
        equation_list.append(decision == estimator_sum + init_value)

        if predicted_class == 0:
            final_equation = decision < 0
        else:
            final_equation = decision > 0

        return final_equation, And(equation_list)

    def instance_expression(self, instance):
        formula = [Real(self.columns[i]) == value for i, value in enumerate(instance)]
        return formula

    def explain_expression(self, I, T_s, D_s, model, reorder):
        i_expression = I.copy()

        importances = model.feature_importances_
        non_zero_indices = np.where(importances != 0)[0]

        if reorder == "asc":
            sorted_feature_indices = non_zero_indices[
                np.argsort(importances[non_zero_indices])
            ]
            i_expression = [i_expression[i] for i in sorted_feature_indices]
        elif reorder == "desc":
            sorted_feature_indices = non_zero_indices[
                np.argsort(-importances[non_zero_indices])
            ]
            i_expression = [i_expression[i] for i in sorted_feature_indices]

        for feature in i_expression.copy():
            # print("\n---removed", feature)
            i_expression.remove(feature)

            # prove(Implies(And(And(i_expression), T), D))
            if self.is_proved(Implies(And(And(i_expression), T_s), D_s)):
                continue
                # print('proved')
            else:
                # print('not proved')
                i_expression.append(feature)
        # print(self.is_proved(Implies(And(And(i_expression), T_s), D_s)))
        return i_expression

    def explain_expression_prob(self, I, T_s, D_s, model, reorder, threshold_margin):
        i_expression = I.copy()

        importances = model.feature_importances_
        non_zero_indices = np.where(importances != 0)[0]

        if reorder == "asc":
            sorted_feature_indices = non_zero_indices[
                np.argsort(importances[non_zero_indices])
            ]
            i_expression = [i_expression[i] for i in sorted_feature_indices]
        elif reorder == "desc":
            sorted_feature_indices = non_zero_indices[
                np.argsort(-importances[non_zero_indices])
            ]
            i_expression = [i_expression[i] for i in sorted_feature_indices]

        threshold = 0
        if threshold_margin != 0:
            threshold = self.predicted_margin * threshold_margin/100
            # print("margin:", self.predicted_margin, "accepted margin:", threshold)
        self.xai_predicted_margin = self.predicted_margin

        for feature in i_expression.copy():
            # print("\n---removed", feature)
            i_expression.remove(feature)

            if self.is_proved_sat(And(And(i_expression), T_s), threshold):
                # print('proved')
                continue
            else:
                # print('not proved -- added back')
                i_expression.append(feature)
        # print(self.is_proved(Implies(And(And(i_expression), T_s), D_s)))
        return i_expression


    def is_proved(self, decision_exp):
        s = Solver()
        s.add(Not(decision_exp))
        if s.check() == unsat:
            return True
        else:
            # print(s.model())
            return False

    def is_proved_sat(self, decision_exp, threshold):
      estimator_sum = Real("estimator_sum")

      debug = Real("debug") == 0
      predicted_class = self.predicted_class

      if predicted_class == 0:
        estmax = Optimize()
        estmax.add(decision_exp)
        estmax.add(debug)
        maxvalue = estmax.maximize(estimator_sum)
        if estmax.check() == sat:
            # print("\nmax sat", maxvalue.value())
            try:
              if float(maxvalue.value().as_fraction()) > threshold:
                  return False # can change class
              else:
                  self.xai_predicted_margin = float(maxvalue.value().as_fraction())
            except:
              print("error max =", maxvalue.value())
              return False
        else:
            print("error")

      if predicted_class == 1:
        estmin = Optimize()
        estmin.add(decision_exp)
        estmin.add(debug)
        minvalue = estmin.minimize(estimator_sum)
        if estmin.check() == sat:
            # print("\nmin sat", minvalue.value())
            try:
              if float(minvalue.value().as_fraction()) < threshold:
                  return False # can change class
              else:
                  self.xai_predicted_margin = float(minvalue.value().as_fraction())
            except:
              print("error min =", minvalue.value())
              return False
        else:
            print("error")

      return True

# main

In [3]:
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target

y[y == 2] = 0  # converte em binario
# X = X.iloc[:, :2] # corta colunas do df

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=101
)

model = XGBClassifier(n_estimators=100, max_depth=3, learning_rate=1)
model.fit(X_train, y_train)

preds = model.predict(X_test)
preds

array([0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 0, 0])

In [4]:
y_pred = model.predict(X_test)
y_pred

array([0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 0, 0])

In [5]:
explainer = XGBoostExplainer(model, X_train)

In [6]:
exp = explainer.explain_prob(X_test.values[0], reorder="desc")
print(exp)

[petal length (cm) == 1.4, petal width (cm) == 0.2]


In [7]:
exp = explainer.explain_prob(X_test.values[0], reorder="desc", threshold_margin=100)
print(exp)
print(explainer.xai_predicted_margin)

[petal length (cm) == 1.4, petal width (cm) == 0.2, sepal width (cm) == 4.2, sepal length (cm) == 5.5]
-3.141518


In [8]:
exp = explainer.explain(X_test.values[0], reorder="desc")
print(exp)

[petal length (cm) == 1.4, sepal width (cm) == 4.2, sepal length (cm) == 5.5]


In [9]:
# print(explainer.T_model)

In [10]:
print(explainer.init_value)

-0.7333348


In [11]:
booster = model.get_booster()

sample = X_test.iloc[[0]]  # Pegamos apenas uma amostra

pred_contrib = booster.predict(xgb.DMatrix(sample), pred_contribs=True)
pred_contrib = np.array(pred_contrib[0])

# print(pred_contrib.shape)
print(pred_contrib)

contribuicoes_somadas = np.sum(pred_contrib[:-1])
print(contribuicoes_somadas)

logits = contribuicoes_somadas + pred_contrib[-1]
print(logits)

bias_column = pred_contrib[-1]
print("Bias term:", bias_column)

[ 0.07500264 -0.04069147 -2.756219    1.0742453  -1.4938543 ]
-1.6476625
-3.1415167
Bias term: -1.4938543


In [12]:
range_metric_list = []
for i in range(X_test.shape[0]):
    print("modelprob:", model.predict([X_test.values[i]], output_margin=True).item())
    print("- xreasson (prob=0):")
    print(explainer.explain_prob(X_test.values[i], reorder="asc"))
    print("xaiprob:",  explainer.xai_predicted_margin)

    t1 = 37
    print(f"\n- threshold {t1}%")
    print(explainer.explain_prob(X_test.values[i], reorder="asc", threshold_margin=t1))
    print("xaiprob:",  explainer.xai_predicted_margin)

    # t2 = 100
    # print(f"\n- threshold {t2}%")
    # print(explainer.explain_prob(X_test.values[i], reorder="asc", threshold_margin=t2))
    # print("xaiprob:", explainer.xai_predicted_margin)
    print("\n-----------------------------")

modelprob: -3.1415181159973145
- xreasson (prob=0):
[petal width (cm) == 0.2, petal length (cm) == 1.4]
xaiprob: -0.23972497837

- threshold 37%
[sepal length (cm) == 5.5, petal width (cm) == 0.2, petal length (cm) == 1.4]
xaiprob: -1.5097533384

-----------------------------
modelprob: -4.220382213592529
- xreasson (prob=0):
[petal width (cm) == 0.4, petal length (cm) == 1.3]
xaiprob: -0.23972497837

- threshold 37%
[sepal length (cm) == 5.4, petal width (cm) == 0.4, petal length (cm) == 1.3]
xaiprob: -1.7848787084

-----------------------------
modelprob: -5.1165947914123535
- xreasson (prob=0):
[petal width (cm) == 0.6, petal length (cm) == 1.6]
xaiprob: -0.23972497837

- threshold 37%
[sepal length (cm) == 5, petal width (cm) == 0.6, petal length (cm) == 1.6]
xaiprob: -2.6810915544

-----------------------------
modelprob: 3.4081568717956543
- xreasson (prob=0):
[petal width (cm) == 1.6, petal length (cm) == 5.8]
xaiprob: 1.45545608013

- threshold 37%
[petal width (cm) == 1.6, pet

# filter df examples

In [14]:
def filter_df_by_conditions(df, conditions):
    """
    Filtra um DataFrame com base em uma lista de condições no formato:
    [feature1 == value1, feature2 == value2, ...]

    Args:
        df (pd.DataFrame): DataFrame original contendo os dados.
        conditions (list): Lista de condições no formato [feature == value, feature == value].
        target_column (str): Nome da coluna alvo (y) a ser mantida no resultado.

    Returns:
        pd.DataFrame: DataFrame filtrado com as condições aplicadas.
    """
    filtered_df = df.copy()
    for condition in conditions:
        feature, value = condition.arg(0), float(condition.arg(1).as_fraction())  # Extrai nome e valor
        filtered_df = filtered_df[filtered_df[str(feature)] == value]  # Filtra o DataFrame

    return filtered_df

In [26]:
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target

y[y == 2] = 0  # converte em binario
# X = X.iloc[:, :2] # corta colunas do df

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=101
)

In [27]:
X

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [28]:
model = XGBClassifier(n_estimators=100, max_depth=3, learning_rate=1)
model.fit(X, y)
explainer = XGBoostExplainer(model, X)

In [29]:
sample = X.values[0]
print(sample)
conditions = explainer.explain_prob(sample, reorder="asc")

[5.1 3.5 1.4 0.2]


In [30]:
for i in range(len(X)):
  sample = X.values[i]
  conditions = explainer.explain_prob(sample, reorder="asc")
  # print(conditions)
  filtered = filter_df_by_conditions(pd.concat([X, pd.DataFrame(y, columns=['target'])], axis=1), conditions)
  if len(filtered) > 3:
    print(i, "| len = ", len(filtered))

51 | len =  5
66 | len =  5
68 | len =  5
78 | len =  5
84 | len =  5


In [31]:
sample = X.values[51]
conditions = explainer.explain_prob(sample, reorder="asc")
print(conditions)
filtered = filter_df_by_conditions(pd.concat([X, pd.DataFrame(y, columns=['target'])], axis=1), conditions)
filtered

[petal length (cm) == 4.5, petal width (cm) == 1.5]


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
51,6.4,3.2,4.5,1.5,1
66,5.6,3.0,4.5,1.5,1
68,6.2,2.2,4.5,1.5,1
78,6.0,2.9,4.5,1.5,1
84,5.4,3.0,4.5,1.5,1


In [35]:
conditions = explainer.explain_prob(sample, reorder="asc", threshold_margin=40)
print(conditions)
filtered = filter_df_by_conditions(pd.concat([X, pd.DataFrame(y, columns=['target'])], axis=1), conditions)
filtered

[sepal length (cm) == 6.4, petal length (cm) == 4.5, petal width (cm) == 1.5]


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
51,6.4,3.2,4.5,1.5,1


## adult dataset

In [62]:
adult_data = fetch_data('adult')
print(adult_data.shape)
adult_data.head()

(48842, 15)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,39.0,7,77516.0,9,13.0,4,1,1,4,1,2174.0,0.0,40.0,39,1
1,50.0,6,83311.0,9,13.0,2,4,0,4,1,0.0,0.0,13.0,39,1
2,38.0,4,215646.0,11,9.0,0,6,1,4,1,0.0,0.0,40.0,39,1
3,53.0,4,234721.0,1,7.0,2,6,0,2,1,0.0,0.0,40.0,39,1
4,28.0,4,338409.0,9,13.0,2,10,5,2,0,0.0,0.0,40.0,5,1


In [73]:
X = adult_data.drop(columns=['target'])
y = adult_data['target'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=101
)

len(X_test)

4885

In [70]:
model = XGBClassifier(n_estimators=30, max_depth=3, learning_rate=1)
model.fit(X, y)
explainer = XGBoostExplainer(model, X)

In [71]:
sample = X.values[0]
print(sample)
conditions = explainer.explain_prob(sample, reorder="asc")
print(conditions)

[3.9000e+01 7.0000e+00 7.7516e+04 9.0000e+00 1.3000e+01 4.0000e+00
 1.0000e+00 1.0000e+00 4.0000e+00 1.0000e+00 2.1740e+03 0.0000e+00
 4.0000e+01 3.9000e+01]
[workclass == 7, capital-loss == 0, hours-per-week == 40, age == 39, capital-gain == 2174, education-num == 13, relationship == 1]


In [81]:
for i in range(len(X_test)):
  sample = X_test.values[i]
  conditions = explainer.explain_prob(sample, reorder="asc")
  # print(conditions)
  filtered = filter_df_by_conditions(pd.concat([X, pd.DataFrame(y, columns=['target'])], axis=1), conditions)
  if len(filtered) >= 8:
    print(i, "| len = ", len(filtered))
    # break

0 | len =  14
3 | len =  8
34 | len =  60
40 | len =  86
44 | len =  87
46 | len =  24
56 | len =  12
57 | len =  8
74 | len =  28
89 | len =  16
94 | len =  22
102 | len =  234
104 | len =  22
111 | len =  9
123 | len =  19
124 | len =  77


KeyboardInterrupt: 

In [89]:
sample = X_test.values[102]
conditions = explainer.explain_prob(sample, reorder="asc")
print(conditions)
filtered = filter_df_by_conditions(pd.concat([X, pd.DataFrame(y, columns=['target'])], axis=1), conditions)
filtered

[sex == 0, occupation == 1, capital-loss == 0, hours-per-week == 40, capital-gain == 0, education-num == 9, relationship == 4]


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
43,49.0,4,94638.0,11,9.0,5,1,4,4,0,0.0,0.0,40.0,39,1
508,47.0,2,543162.0,11,9.0,5,1,4,2,0,0.0,0.0,40.0,39,1
646,31.0,2,402089.0,11,9.0,0,1,4,4,0,0.0,0.0,40.0,39,1
987,47.0,4,335973.0,11,9.0,0,1,4,4,0,0.0,0.0,40.0,39,1
1178,47.0,4,45857.0,11,9.0,4,1,4,4,0,0.0,0.0,40.0,39,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48486,57.0,4,113974.0,11,9.0,0,1,4,4,0,0.0,0.0,40.0,39,1
48548,29.0,4,67306.0,11,9.0,4,1,4,0,0,0.0,0.0,40.0,39,1
48586,44.0,4,216116.0,11,9.0,0,1,4,2,0,0.0,0.0,40.0,23,1
48754,34.0,4,31341.0,11,9.0,0,1,4,4,0,0.0,0.0,40.0,39,1


In [91]:
filtered.target.value_counts()

target
1    233
0      1
Name: count, dtype: int64

In [99]:
conditions = explainer.explain_prob(sample, reorder="asc", threshold_margin=27)
print(conditions)
filtered = filter_df_by_conditions(pd.concat([X, pd.DataFrame(y, columns=['target'])], axis=1), conditions)
filtered

[marital-status == 0, workclass == 4, sex == 0, occupation == 1, capital-loss == 0, hours-per-week == 40, capital-gain == 0, education-num == 9, relationship == 4]


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
987,47.0,4,335973.0,11,9.0,0,1,4,4,0,0.0,0.0,40.0,39,1
1228,50.0,4,358740.0,11,9.0,0,1,4,4,0,0.0,0.0,40.0,9,1
1313,51.0,4,179646.0,11,9.0,0,1,4,4,0,0.0,0.0,40.0,39,1
1752,37.0,4,349689.0,11,9.0,0,1,4,4,0,0.0,0.0,40.0,39,1
3325,39.0,4,172718.0,11,9.0,0,1,4,4,0,0.0,0.0,40.0,39,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46070,51.0,4,29036.0,11,9.0,0,1,4,4,0,0.0,0.0,40.0,39,1
47200,28.0,4,156967.0,11,9.0,0,1,4,4,0,0.0,0.0,40.0,39,1
48486,57.0,4,113974.0,11,9.0,0,1,4,4,0,0.0,0.0,40.0,39,1
48586,44.0,4,216116.0,11,9.0,0,1,4,2,0,0.0,0.0,40.0,23,1


In [98]:
filtered.target.value_counts()

target
1    1
Name: count, dtype: int64